# 1- Packages

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from scipy import stats
from textwrap import wrap
import seaborn as sns
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score


%matplotlib inline

# 2 - Data acquisition

In [442]:
Europe=pd.read_excel(r'C:\Users\Eric.Li\Documents\IPO data\IPO.xlsx',sheet_name="Europe").set_index("Ticker")
price_Europe=pd.read_excel(r'C:\Users\Eric.Li\Documents\IPO data\IPO.xlsx',sheet_name="Europe price").set_index("Ticker")
index_Europe=pd.read_excel(r'C:\Users\Eric.Li\Documents\IPO data\IPO.xlsx',sheet_name="Europe index").set_index("Ticker")

In [443]:
def get_return(price_df,index_df):
    abs_return=price_df.diff(1,axis=1)/price_df.shift(1,axis=1)
    index_return=index_df.diff(1,axis=1)/index_df.shift(1,axis=1)
    rel_return=abs_return-index_return
    return abs_return,rel_return

In [444]:
abs_return_Europe,rel_return_Europe=get_return(price_Europe,index_Europe)

# 3 - Strategy

In [445]:
def IPO_long(data_df,price_df,return_df,Size,Open,Volume,liquidity_constraint,start_day,end_day,holding):
    target_data=data_df[(data_df["Market_cap"]>=Size)&(data_df["Offer_Day1_Open"]>Open)&(data_df["Volume_shares"]>Volume)&(data_df["5d_liquidity"]>liquidity_constraint)]
    target_price=price_df.loc[target_data.index.tolist()]
    high_evaluate=(target_price.expanding(min_periods=start_day,axis=1).max()==target_price).iloc[:,start_day-1:end_day]
    high_evaluate=high_evaluate[high_evaluate==True].dropna(how='all',axis=0)
    trade_day=high_evaluate.apply(lambda x:x.tolist().index(1.0),axis=1)    
    target_return=return_df.loc[trade_day.index]
    signal_df=target_return.apply(lambda x:x[5-1+trade_day[x.name]:5-1+trade_day[x.name]+holding],axis=1).shift(1,axis=1)
    signal_df["Index"]=signal_df.apply(lambda x:(x.name,data_df["IPO_date"].loc[x.name].date(),trade_day[x.name]+6),axis=1)
    signal_df=signal_df.set_index("Index")
    return signal_df

In [516]:
signal_Europe=IPO_long(Europe,price_Europe,rel_return_Europe,300,-1,0.1,5,5,100,40)

In [517]:
signal_Europe.sum(axis=1).mean()

0.06337397644381432

In [518]:
signal_Europe.shape

(213, 110)

In [480]:
signal_Europe

,5,6,7,8,9,10,11,12,13,14,...,140,141,142,143,144,145,146,147,148,149
Index,,,,,,,,,,,,,,,,,,,,,
"(GLEN LN Equity, 2011-05-19, 10)",NaN,NaN,NaN,NaN,NaN,-0.002777,-0.020414,0.010107,-0.013897,0.028327,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(AENA SM Equity, 2015-02-11, 6)",NaN,0.031606,0.018345,-0.002787,0.010316,-0.011272,-0.011290,0.009629,0.022763,0.023280,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(CLNX SM Equity, 2015-05-07, 6)",NaN,0.032011,0.005598,-0.017237,-0.012303,0.004870,-0.000361,-0.006517,-0.022294,0.001198,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(RMG LN Equity, 2013-10-11, 7)",NaN,NaN,0.039260,-0.007418,-0.003646,0.085546,-0.022123,0.050617,-0.011209,-0.031422,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(STR AV Equity, 2007-10-18, 9)",NaN,NaN,NaN,NaN,0.032766,0.002681,0.020302,0.016598,0.013361,-0.016131,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(AMS SM Equity, 2010-04-29, 12)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.055415,0.030948,0.019950,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(IGY GR Equity, 2016-10-07, 7)",NaN,NaN,-0.002306,0.011222,0.006851,-0.015989,-0.011102,-0.005748,-0.009091,-0.000038,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(PIRC IM Equity, 2017-10-04, 21)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(RLIA SM Equity, 2007-06-06, 21)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
